In [1]:
import pandas as pd
import numpy as np
import os

## En datos/properati deben estar todos los archivos de ventas provistos por properati

In [2]:
filesProperati = filter( lambda f: not f.startswith('.'), os.listdir("datos/properati"))

In [3]:
dataframes = []

for file_name in filesProperati:
    
    newDataFrame = pd.read_csv("datos/properati/"+file_name, error_bad_lines = False)
    dataframes.append(newDataFrame)


## ¿Como filtramos por las zonas pedidas?

In [4]:
nulosStateName = 0
nulosPlaceWParentNames = 0
noCuentaConStateName = 0
noCuentaConPlaceWParentNames = 0
for df in dataframes:
    try:
        nulosStateName += df.state_name.isnull().sum()
    except:
        noCuentaConStateName += 1
    try:
        nulosPlaceWParentNames += df.place_with_parent_names.isnull().sum()
    except:
        noCuentaConPlaceWParentNames += 1
print "Cantidad de registros con valor nulo en state_name: "+str(nulosStateName)
print "Cantidad de registros con valor nulo en place_w_parent_names: "+str(nulosPlaceWParentNames)
print "Cantidad de archivos que no cuentan con la columna state_name: "+str(noCuentaConStateName)
print "Cantidad de archivos que no cuentan con la columna place_w_parent_names: "+str(noCuentaConPlaceWParentNames)

Cantidad de registros con valor nulo en state_name: 0
Cantidad de registros con valor nulo en place_w_parent_names: 0
Cantidad de archivos que no cuentan con la columna state_name: 26
Cantidad de archivos que no cuentan con la columna place_w_parent_names: 0


## Dado los resultados, para quedarnos solo con las propiedades de Capital Federal y GBA tendremos que utilizar la columna 'place_with_parent_names'

In [5]:
frecuencias = {}
for df in dataframes:
    parentNames = df['place_with_parent_names']
    for padres in parentNames:
        zona = padres.split("|")[2]
        if zona not in frecuencias:
            frecuencias[zona] = 1
        else:
            frecuencias[zona]+=1

In [6]:
for zona in frecuencias: 
    print zona,frecuencias[zona]

San Juan 649
Córdoba 117036
Chaco 1976
Mendoza 9302
Colonia 205
Chubut 1637
Capital Federal 699655
San Luis 7437
Tucumán 4774
Misiones 11996
Tierra Del Fuego 941
Bs.As. G.B.A. Zona Oeste 327481
Corrientes 2064
Formosa 408
Maldonado 6101
Río Negro 18831
Santa Cruz 285
New York 41
California 29
Montevideo 3281
Neuquén 12472
La Pampa 5440
Buenos Aires Costa Atlántica 322874
Buenos Aires Interior 68078
Miami 987
Florida 1187
Bs.As. G.B.A. Zona Sur 185564
Santa Catarina 268
Santiago Del Estero 286
Rocha 85
Jujuy 535
Punta del Este 15209
Santa Fe 132237
Salta 7969
La Rioja 341
Entre Ríos 16943
Bs.As. G.B.A. Zona Norte 585113
Catamarca 1015


## Interesan: Bs.As. G.B.A. Zona Norte, Bs.As. G.B.A. Zona Sur, Bs.As. G.B.A. Zona Oeste, Capital Federal


In [7]:
def filtrarCapitalYGBA(serie):
    deseados = ["Bs.As. G.B.A. Zona Norte", "Bs.As. G.B.A. Zona Sur", "Bs.As. G.B.A. Zona Oeste", "Capital Federal"]
    booleans = []
    for item in serie:
        if item.split("|")[2] in deseados:
            booleans.append(True)
        else:
            booleans.append(False)
    return booleans

In [8]:
filtrados =[]
for df in dataframes:
    
    df = df[filtrarCapitalYGBA(df['place_with_parent_names'])]
    filtrados.append(df)

### Por las dudas chequeo que ahora las frecuencias sean las mismas y solo queden las zonas que interesan

In [9]:
frecuencias = {}
for df in filtrados:
    parentNames = df['place_with_parent_names']
    for padres in parentNames:
        zona = padres.split("|")[2]
        if zona not in frecuencias:
            frecuencias[zona] = 1
        else:
            frecuencias[zona]+=1

In [10]:
for key in frecuencias:
    print key, frecuencias[key]

Capital Federal 699655
Bs.As. G.B.A. Zona Norte 585113
Bs.As. G.B.A. Zona Oeste 327481
Bs.As. G.B.A. Zona Sur 185564


# Ahora recupero datos, elimino algunas columnas, y demas.
# Primero miro como estan los datos, para ver si conviene trabajar con los precios en pesos argentinos o dolares. 

In [11]:
def a(df, column_name):
    try:
        return(df[column_name].isnull().sum(),0)
    except:
        return (0,1)

In [12]:
nulsPriceAproxLocalCurr = 0
nulsPriceAproxUsd = 0
nulsPricePerM2 = 0
nulsPricePerM2usd = 0

priceAproxLocalCurrInex = 0
priceAproxUsdInex = 0
pricePerM2Inex = 0
pricePerM2usdInex = 0

for df in filtrados:
    nuls, inex = a(df, "price_aprox_local_currency")
    nulsPriceAproxLocalCurr += nuls
    priceAproxLocalCurrInex += inex
    nuls,inex = a(df, "price_aprox_usd")
    nulsPriceAproxUsd += nuls
    priceAproxUsdInex += inex
    nuls, inex = a(df, "price_per_m2")
    nulsPricePerM2 += nuls
    pricePerM2Inex += inex
    nuls,inex = a(df, "price_usd_per_m2")
    nulsPricePerM2usd += nuls
    pricePerM2usdInex += inex

In [13]:
print str(nulsPriceAproxLocalCurr)+" Nulls priceAproxLocalCurr"
print str(nulsPriceAproxUsd)+" Nulls priceAproxUsd"
print str(nulsPricePerM2)+" Nulls Price Per M2"
print str(nulsPricePerM2usd)+" Nulls pricePerM2USD"
print str(priceAproxLocalCurrInex)+" Inexistentes priceAproxLocalCurr"
print str(priceAproxUsdInex)+" Inexistentes priceAproxUsd"
print str(pricePerM2Inex)+" Inexistentes pricePerM2"
print str(pricePerM2usdInex)+" Inexistentes pricePerM2usd"

225688 Nulls priceAproxLocalCurr
225688 Nulls priceAproxUsd
116516 Nulls Price Per M2
674702 Nulls pricePerM2USD
0 Inexistentes priceAproxLocalCurr
0 Inexistentes priceAproxUsd
26 Inexistentes pricePerM2
0 Inexistentes pricePerM2usd


# Tambien miro los datos acerca de las superficies

## Observo que algunos archivos solo tienen una columna 'surface_in_m2', no distingue entre si es cubierta o si es total, suponemos que es la superficie total. 

In [14]:
nulsSuperficieTotal = 0
for df in filtrados:
    try:
        nulsSuperficieTotal += df.surface_in_m2.isnull().sum()
    except:
        nulsSuperficieTotal += df.surface_total_in_m2.isnull().sum()
nulsSuperficieTotal

475805

In [15]:
#Cuenta cuantas propiedades tienen el precio por m2 en dolares nulo, 
#pero el precio local no nulo
count = 0
for df in filtrados:
    try:
        count += len(df[~df.price_per_m2.isnull() & df.price_usd_per_m2.isnull()])
    except:
        #Algunos dataframes no tienen el precio por m2 en pesos
        continue
print "Cantidad de propiedades de las que se conoce el precio por m2 (En pesos)\
 pero no en dolares: "+str(count)

Cantidad de propiedades de las que se conoce el precio por m2 (En pesos) pero no en dolares: 107423


# ¿La columna price, tiene el mismo precio que price_aprox_usd o el mismo precio que price_aprox_local_curr? solo analizamos para el caso en el que no este el precio en dolares

In [16]:
mismo_usd = 0
mismo_aprox_local_curr = 0
for df in filtrados:
    mismo_aprox_local_curr += len(df[(df["price_aprox_local_currency"] == df["price"]) &\
                                    (df["price"] != df["price_aprox_usd"]) & \
                                    ~df["price_aprox_usd"].isnull()])
    mismo_usd += len(df[df["price_aprox_usd"] == df["price"]])

print mismo_aprox_local_curr
print mismo_usd
    

88
1433441


# Salvo pocos casos, la columna price contiene los mismos valores que la columna price_aprox_usd. Descarto la columna price. ¿Puedo recuperar precio de propiedades en dolares a partir del precio aproximado local?

In [17]:
coincidencias = 0
for df in filtrados:
    coincidencias += len(df[(~df["price_aprox_local_currency"].isnull() & df["price_aprox_usd"].isnull()) |\
                            (~df["price"].isnull() & df["price_aprox_usd"].isnull())])

print coincidencias
    

2


# Son tan solo dos casos en los que el precio aproximado local no es nulo y el precio en dolares lo es. No tenerlo en cuenta no deberia afectarnos

# Se pueden recuperar datos de los precios en dolares por m2 realizando la correspondiente conversion. De igual manera, antes estudio para ver que no se pueda recuperar de otra manera. 

## Antes de realizar conversiones o algunos calculos para poder recuperar el precio en USD, investigo si con la descripcion puedo recuperar algo de esto

In [18]:
sum(len(x) for x in filtrados) #cuantos registros tenemos en total?

1797813

In [19]:
for df in filtrados:
    if "description" in df.columns:
        df["description"] = df["description"].str.lower()
    if "title" in df.columns:
        df["title"] = df["title"].str.lower()
    if "extra" in df.columns:
        df["extra"] = df["extra"].str.lower()

/home/tomas/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/tomas/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/tomas/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [20]:
total_no_price_usd = 0 
total_recuperable_info = 0
for df in filtrados:
    df_no_price_usd = df[df["price_aprox_usd"].isnull()]
    cantidad = len(df_no_price_usd)
    cols_considerar = []
    if "extra" in df.columns:
        cols_considerar.append("extra")
    if "title" in df.columns:
        cols_considerar.append("title")
    if "description" in df.columns:
        cols_considerar.append("description")
        
    posibles_recuperables = 0
    for col in cols_considerar:
        
        menciona_dolares = df_no_price_usd[df_no_price_usd[col].str.contains("dolares")]
        menciona_dolares_2 = df_no_price_usd[df_no_price_usd[col].str.contains("dólares")]
        menciona_usd = df_no_price_usd[df_no_price_usd[col].str.contains("usd")]
        posibles_recuperables += (len(menciona_dolares) + len(menciona_dolares_2) + len(menciona_usd))
    total_no_price_usd += cantidad
    total_recuperable_info += posibles_recuperables
print "Se encontro que "+str(total_no_price_usd)+" registros no tienen informacion del precio en USD"
print "Se encontro tambien que en "+str(total_recuperable_info)+" registros se podria recuperar el precio en USD de otras columnas"
print ""

Se encontro que 225688 registros no tienen informacion del precio en USD
Se encontro tambien que en 25603 registros se podria recuperar el precio en USD de otras columnas



## Hay algunos precios en dolar que podemos recuperar de la descripcion! hacemos una vista previa a el sector de la descripcion donde se menciona USD o dolares

In [21]:
for df in filtrados:
    df_no_price_usd = df[df["price_aprox_usd"].isnull()]
    cantidad = len(df_no_price_usd)
    cols_considerar = []
    if "extra" in df.columns:
        cols_considerar.append("extra")
    if "title" in df.columns:
        cols_considerar.append("title")
    if "description" in df.columns:
        cols_considerar.append("description")
        
    posibles_recuperables = 0
    for col in cols_considerar:
        
        menciona_dolares = df_no_price_usd[df_no_price_usd[col].str.contains("dolares")].head(1)
        menciona_dolares_2 = df_no_price_usd[df_no_price_usd[col].str.contains("dólares")].head(1)
        menciona_usd = df_no_price_usd[df_no_price_usd[col].str.contains("usd")].head(1)
        dfs = [menciona_dolares, menciona_dolares_2, menciona_usd]
        for d in dfs:
            for index, row in d.iterrows():
                text= row['description']
                print ""
                print text
        break


codigo: 779-abc555 ubicado en: 9 de julio 400 -  publicado por: c. gutierrez operaciones inmobiliarias. el precio es de ars 68000 null. epartamento a estrenar de 2 ambientes al frente con cochera. características: 41 m² cubiertos, balcón de 3 m², cochera, aberturas de aluminio, instalación para aire acondicionado, conexión para lavarropa, termotanque y anafe eléctrico, cocina con bajo mesada y alacena, pisos de porcelanato. bajas expensas. . publicado a través de mapaprop

 <br>venta departamento en caballito  ubicado en una excelente zona residencial del barrio de caballito. donde coinciden y se complementan el centro geogrÃ¡fico de la ciudad y el parque centenario, una de las Ã¡reas verdes mÃ¡s significativas de todo buenos aires. una zona residencial en constante crecimiento y revalorizaciÃ³n, con fÃ¡cil acceso y generosa en medios de transporte pÃºblicos.   el proyecto contempla 35 unidades de entre 37 y 41,5 m2 con una calidad en el diseÃ±o arquitectÃ³nico que maximiza el uso y l


alquiler amoblado - 4 pasajerossan telmo - puerto madero - centro buenos airesalquiler departamento amueblado.luminoso departamento, ubicado a dos cuadras de la avenida alicia moreau de justo, lujoso barrio de puerto madero, zona repleta de restaurantes y bares junto al río. este elegante departamento es ideal para dos personas y cuenta con: un dormitorio, un living-comedor, una cocina con barra desayunadora con dos banquetas, un baño completo, un toilette para visitas y un hermoso balcón aterrazado al que se accede por el comedor y el dormitorio. además, el edificio posee lobby con servicio recepción y vigilancia las 24hs, dos salas de estar, biblioteca, jardín, piscina, solarium, gimnasio. en unos meses se podrá utilizar el salón de fiestas y estacionamiento de automóviles opcional. el edificio cuenta con: -piscina -gym -servicio de vigilancia las 24hs. -solarium -sum / salón de fiestas -lobby con biblioteca alquiler por 2 aÑos $24.700 para los casos de alquiler de vivienda a person

# Lamentablemente se observan cosas del estilo '1 usd null' o precios concatenados con otros strings, precios en distintos formatos y ademas a veces se menciona dolares/usd y se podria estar hablando de otra cosa, si bien se podria hacer el trabajo seria muy arduo y no vale la pena en este caso para  25000 registros de un total de 1797813.. de todos modos algunos de estos precios podrian ser recuperados posteriormente de otras formas que iremos viendo a continuacion: 

# ¿A cuantos se les desconoce el precio por m2 en USD y no se conoce el precio total (en USD) o la superficie total?

In [22]:
  coincidencias = 0
for df in filtrados:
    if "price_per_m2" in df.columns:
        coincidencias += len(df[(~df["price_per_m2"].isnull() & df["price_usd_per_m2"].isnull())\
                           & (df["price_aprox_usd"].isnull() | df["surface_total_in_m2"].isnull())])

print coincidencias  

94025


# 94025 de 639946 registros pueden ser recuperados si realizamos la conversion 

# Los datos para las conversiones fueron encontrados en:
### www.bcra.gob.ar/Pdfs/PublicacionesEstadisticas/com3500.xls
# Los mismos fueron formateados de manera conveniente por como se nos presentan los datos de properati, usando LibreOffice Calc

In [23]:

datosDolar = pd.read_csv("info_dolar.csv",index_col = "Fecha")

In [24]:
from datetime import datetime
from datetime import timedelta
def recuperar_precio_m2_en_dolares(rowProperati):
    if ~pd.isnull(rowProperati["price_per_m2"]) & pd.isnull(rowProperati["price_usd_per_m2"]):
        fechaCreacionPropiedad = rowProperati["created_on"]
        anho = int(fechaCreacionPropiedad.split("-")[0])
        mes = int(fechaCreacionPropiedad.split("-")[1])
        dia = int(fechaCreacionPropiedad.split("-")[2])
        fechaConCotizacion = datetime(anho, mes, dia)
        fechaEncontrada = False
        while not(fechaEncontrada):
            try:
                cotizFechaCreacProp = float(datosDolar.ix[fechaConCotizacion.strftime('%Y-%m-%d')]["Cotizacion Dolar"])
                fechaEncontrada = True
            except:
                #En esa fecha no se habia publicado informacion del dolar (no vario)
                #Me fijo en el dia anterior
                fechaConCotizacion = fechaConCotizacion - timedelta(days = 1)
        return cotizFechaCreacProp*rowProperati["price_per_m2"]
    else:
        return rowProperati["price_per_m2"]

In [25]:
for df in filtrados:
    if "price_per_m2" in df.columns:
        df["price_usd_per_m2"] = df.apply(lambda row: recuperar_precio_m2_en_dolares(row),axis=1)

/home/tomas/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  del sys.path[0]
/home/tomas/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# Corroboro

In [26]:
coincidencias = 0
for df in filtrados:
    if "price_per_m2" in df.columns:
        coincidencias += len(df[(~df["price_per_m2"].isnull() & df["price_usd_per_m2"].isnull())\
                           & (df["price_aprox_usd"].isnull() | df["surface_total_in_m2"].isnull())])

print coincidencias    

0


# Se recuperaron 94025 registros mediante la conversion a dolares del precio del metro cuadrado

# ¿Se puede recuperar mas informacion?

In [27]:
#Primero le pongo el mismo nombre a la superficie total
for df in filtrados:
    try:
        df['surface_total_in_m2']
    except:
        df.rename(columns={'surface_in_m2':'surface_total_in_m2'},inplace=True)
    

/home/tomas/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:2844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [28]:
#S 'se tiene informacion de' N 'no se tiene informacion de'
S_ppm2_S_sup_N_precio = 0
S_ppm2_S_precio_N_sup = 0
S_sup_S_precio_N_ppm2 = 0
for df in filtrados:
    S_ppm2_S_sup_N_precio += len(df[~df["price_usd_per_m2"].isnull() &\
                                    ~df["surface_total_in_m2"].isnull() &\
                                    df["price_aprox_usd"].isnull()])
    S_ppm2_S_precio_N_sup += len(df[~df["price_usd_per_m2"].isnull() &\
                                    ~df["price_aprox_usd"].isnull() &\
                                    df["surface_total_in_m2"].isnull()])
    S_sup_S_precio_N_ppm2 += len(df[~df["surface_total_in_m2"].isnull() &\
                                    ~df["price_aprox_usd"].isnull() &\
                                    df["price_usd_per_m2"].isnull()])
print S_ppm2_S_sup_N_precio
print S_ppm2_S_precio_N_sup
print S_sup_S_precio_N_ppm2

278
94025
56040


# Recupero esta informacion

In [29]:
from utils_recuperacion_datos import recuperar_superficie,recuperar_precio_usd,recuperar_ppm2

In [30]:
for df in filtrados:
     df['surface_total_in_m2'] = df.apply(lambda row: recuperar_superficie(row),axis=1)
     df['price_aprox_usd'] = df.apply(lambda row: recuperar_precio_usd(row),axis=1)
     df['price_usd_per_m2'] = df.apply(lambda row: recuperar_ppm2(row),axis=1)

/home/tomas/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/tomas/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/tomas/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


## Chequeo que se hayan recuperado datos

In [31]:
#S 'se tiene informacion de' N 'no se tiene informacion de'
S_ppm2_S_sup_N_precio = 0
S_ppm2_S_precio_N_sup = 0
S_sup_S_precio_N_ppm2 = 0
for df in filtrados:
    S_ppm2_S_sup_N_precio += len(df[~df["price_usd_per_m2"].isnull() &\
                                    ~df["surface_total_in_m2"].isnull() &\
                                    df["price_aprox_usd"].isnull()])
    S_ppm2_S_precio_N_sup += len(df[~df["price_usd_per_m2"].isnull() &\
                                    ~df["price_aprox_usd"].isnull() &\
                                    df["surface_total_in_m2"].isnull()])
    S_sup_S_precio_N_ppm2 += len(df[~df["surface_total_in_m2"].isnull() &\
                                    ~df["price_aprox_usd"].isnull() &\
                                    df["price_usd_per_m2"].isnull()])
print S_ppm2_S_sup_N_precio
print S_ppm2_S_precio_N_sup
print S_sup_S_precio_N_ppm2

0
0
0


## A continuacion unifico y voy eliminando duplicados, teniendo en cuenta distintos subgrupos de columnas

In [32]:
unificacion = pd.concat(filtrados, axis=0, ignore_index=True)

In [33]:
len(unificacion)

1797813

In [34]:
#Elimino los registros que tengan todas las columnas identicas
a = unificacion.drop_duplicates(keep = "first")

In [35]:
len(a)

1465413

In [36]:
#Los que no tienen ni precio aproximado en dolares  ya no me interesan
b = a[~a["price_aprox_usd"].isnull()]

In [37]:
len(b)

1322101

In [38]:
links_repetidos = b["properati_url"].value_counts()

# Observo un poco que informacion encuentro en estas publicaciones con links repetidos

In [39]:
links_repetidos.head(2)

http://www.properati.com.ar/8mpk_venta_departamento_olivos_avenida-del-libertador-gral-san-martin_3000    23
http://www.properati.com.ar/7f8h_venta_departamento_tres-de-febrero                                       22
Name: properati_url, dtype: int64

In [40]:
b[b["properati_url"] == "http://www.properati.com.ar/95fn_venta_departamento_capital-federal"].head(5)

,,country_name,created_on,currency,description,expenses,extra,floor,geonames_id,id,...,price_aprox_usd,price_per_m2,price_usd_per_m2,properati_url,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2,title
74702,NaN,NaN,2013-08-12,USD,NaN,NaN,NaN,11.0,3433955.0,NaN,...,117900.0,NaN,1871.428571,http://www.properati.com.ar/95fn_venta_departa...,apartment,1.0,NaN,NaN,63.0,NaN
137024,NaN,NaN,2013-08-12,USD,NaN,NaN,NaN,11.0,3433955.0,NaN,...,117900.0,NaN,1871.428571,http://www.properati.com.ar/95fn_venta_departa...,apartment,1.0,NaN,NaN,63.0,NaN
207175,NaN,NaN,2013-08-12,USD,NaN,NaN,NaN,11.0,3433955.0,NaN,...,117900.0,NaN,1871.428571,http://www.properati.com.ar/95fn_venta_departa...,apartment,1.0,NaN,NaN,63.0,NaN
236474,NaN,NaN,2013-08-12,USD,NaN,NaN,NaN,11.0,3433955.0,NaN,...,117900.0,NaN,1871.428571,http://www.properati.com.ar/95fn_venta_departa...,apartment,1.0,NaN,NaN,63.0,NaN
267336,NaN,NaN,2013-08-12,USD,NaN,NaN,NaN,11.0,3433955.0,NaN,...,117900.0,NaN,NaN,http://www.properati.com.ar/95fn_venta_departa...,apartment,1.0,NaN,NaN,NaN,NaN


In [41]:
b[b["properati_url"] == "http://www.properati.com.ar/7v2b_venta_departamento_olivos_avenida-del-libertador-gral-san-martin_2300"].head(5)

,,country_name,created_on,currency,description,expenses,extra,floor,geonames_id,id,...,price_aprox_usd,price_per_m2,price_usd_per_m2,properati_url,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2,title
69454,NaN,NaN,2013-06-14,USD,NaN,NaN,NaN,NaN,3430310.0,NaN,...,192000.0,NaN,4085.106383,http://www.properati.com.ar/7v2b_venta_departa...,apartment,2.0,NaN,NaN,47.0,NaN
130136,NaN,NaN,2013-06-14,USD,NaN,NaN,NaN,NaN,3430310.0,NaN,...,192000.0,NaN,4085.106383,http://www.properati.com.ar/7v2b_venta_departa...,apartment,2.0,NaN,NaN,47.0,NaN
195104,NaN,NaN,2013-06-14,USD,NaN,NaN,NaN,NaN,3430310.0,NaN,...,185000.0,NaN,3936.170213,http://www.properati.com.ar/7v2b_venta_departa...,apartment,2.0,NaN,NaN,47.0,NaN
227790,NaN,NaN,2013-06-14,USD,NaN,NaN,NaN,NaN,3430310.0,NaN,...,192000.0,NaN,4085.106383,http://www.properati.com.ar/7v2b_venta_departa...,apartment,2.0,NaN,NaN,47.0,NaN
265791,NaN,NaN,2013-06-14,USD,NaN,NaN,NaN,6.0,3430310.0,NaN,...,195000.0,NaN,NaN,http://www.properati.com.ar/7v2b_venta_departa...,apartment,2.0,NaN,NaN,NaN,NaN


# Observo que hay publicaciones con misma URL y que parecen ser las mismas propiedades, pero el inconveniente es que los campos vacios de cada uno de estos registros varia, debo filtrar de una forma mas compleja

# Antes de hacer esto voy a eliminar columnas que no interesan

In [42]:
len(b[~b["lat"].isnull() & ~b["lon"].isnull() & b["lat-lon"].isnull()])

0

In [43]:
#Puedo eliminar latitud y longitud, ya que lat-lon tiene su informacion
#tambien, country_name, image_thumbnail, operation, price, price aprox local curr, currency
#ya que solo trabajaremos con precios en dolares


In [44]:
b.columns

Index([u' ', u'country_name', u'created_on', u'currency', u'description',
       u'expenses', u'extra', u'floor', u'geonames_id', u'id',
       u'image_thumbnail', u'lat', u'lat-lon', u'lon', u'operation',
       u'place_name', u'place_with_parent_names', u'price',
       u'price_aprox_local_currency', u'price_aprox_usd', u'price_per_m2',
       u'price_usd_per_m2', u'properati_url', u'property_type', u'rooms',
       u'state_name', u'surface_covered_in_m2', u'surface_total_in_m2',
       u'title'],
      dtype='object')

In [51]:
data = b.drop(["country_name","lat","lon","image_thumbnail",\
               "operation","price",'price_per_m2',"price_aprox_local_currency","currency"],axis=1)

In [63]:
len(data[data.duplicated(subset=["property_type","properati_url","price_aprox_usd","price_usd_per_m2","rooms","floor","place_name","surface_covered_in_m2","surface_total_in_m2","lat-lon","expenses","description","title","extra"])])

476199

In [65]:
#Elimino registros con misma url de properati, mismo precio y misma cantidad de habitaciones y piso 
data = data.drop_duplicates(subset=["property_type","properati_url","price_aprox_usd","price_usd_per_m2","rooms","floor","place_name","surface_covered_in_m2","surface_total_in_m2","lat-lon","expenses","description","title","extra"])

In [66]:
len(data)

845902

# Vamos a intentar recuperar el piso y la cantidad de habitaciones de los registros que no guarden esta informacion pero que la guarden en su descripcion

In [67]:
sin_rooms = data[data["rooms"].isnull()]

In [68]:
print len(sin_rooms)
print len(sin_rooms["description"].str.contains("habitaciones"))
print len(sin_rooms["description"].str.contains("ambiente"))

354982
354982
354982


### capaz podemos llegar a recuperar todas las cantidades de habitaciones!

### Como se presenta esta informacion?

In [69]:
descripciones = sin_rooms["description"].values
print len(sin_rooms["title"].str.contains("ambientes"))

354982


In [70]:
posibles_descripciones_ambientes = {"1 ambiente": 1, "monoambiente": 1, "mono ambiente":1, "2 ambientes": 2,"3 ambientes": 3,
                          "4 ambientes": 4,"5 ambientes": 5, "6 ambientes": 6, "7 ambientes": 7,"8 ambientes": 8,
                          "9 ambientes": 1, "10 ambientes": 10,
                         "ambientes: 1": 1, "ambientes: 2": 2, "ambientes: 3": 3, "ambientes: 4": 4, "ambientes: 5": 5,
                         "ambientes: 6": 6, "ambientes: 7": 7, "un ambiente": 1, "dos ambientes": 2, "tres ambientes": 3,
                         "cuatro ambientes": 4,"cinco ambientes": 5," seis ambientes": 6, "siete ambientes": 7,
                         "2 impecables ambientes": 2, "3 impecables ambientes": 3, "4 amplios ambientes": 4, "2 amplios ambientes": 2,
                          "3 amplios ambientes": 3, "5 amplios ambientes": 5, "2  ambientes": 2, "3  ambientes": 3,
                          "2 grandes ambientes": 2, "6 amplios ambientes": 6, "1 ambientes": 1
                         }

encuentros = 0
for d in descripciones:
    found = False
    for k in posibles_descripciones_ambientes:
        if not(pd.isnull(d)):
            if k in d:
                encuentros += 1
                found = True
                break
#    if not(found) and not(pd.isnull(d)):
#        split = d.split(" ")
#        for i in range(0,len(split)):
#            if split[i] == "ambientes":
#                try:
#                    string = [split[i-2], split[i-1], split[i], split[i+1], split[i + 2]]
#                    for strr in string:
#                        if strr.isdigit():
#                            print " ".join(string)
#                except:
#                    print d
                
    
print encuentros
    

91039


# vemos que podemos recuperar al menos 65727 registros (porque esto no solo lo aplicamos a descripcion, a titulo y extra tambien)  de los que no se tiene su habitacion asignada. Se podrian buscar mas casos extraños para agregar al diccionario pero al parecer no cambian demasiado el total de registros que se pueden recuperar.

## Hago lo mismo para el piso

In [71]:
sin_piso = data[data["floor"].isnull()]
print len(sin_piso)
print len(sin_piso["description"].str.contains("piso"))
descripciones = sin_piso["description"]

689112
689112


In [72]:

posibles_descripciones_piso = {'1° piso': 1,
'2° piso': 2,
'3° piso': 3,
'4° piso': 4,
'5° piso': 5,
'6° piso': 6,
'7° piso': 7,
'8° piso': 8,
'9° piso': 9,
'10° piso': 10,
'11° piso': 11,
'12° piso': 12,
'13° piso': 13,
'14° piso': 14,
'15° piso': 15,
'16° piso': 16,
'17° piso': 17,
'18° piso': 18,
'19° piso': 19,
'20° piso': 20,
'21° piso': 21,
'22° piso': 22,
'23° piso': 23,
'24° piso': 24,
'25° piso': 25,
'26° piso': 26,
'27° piso': 27,
'28° piso': 28,
'29° piso': 29,
'30° piso': 30,
'31° piso': 31,
'32° piso': 32,
'33° piso': 33,
'34° piso': 34,
'35° piso': 35,
'36° piso': 36,
'37° piso': 37,
'38° piso': 38,
'39° piso': 39,
'40° piso': 40,
'41° piso': 41,
'42° piso': 42,
'43° piso': 43,
'44° piso': 44,
'45° piso': 45,
'46° piso': 46,
'47° piso': 47,
'48° piso': 48,
'49° piso': 49,
'50° piso': 50}
encuentros = 0
for d in descripciones:
    found = False
    for k in posibles_descripciones_piso:
        if not(pd.isnull(d)):
            if k in d:
                encuentros += 1
                found = True
                break
#    if not(found) and not(pd.isnull(d)):
#        split = d.split(" ")
#        for i in range(0,len(split)):
#            if split[i] == "piso":
#                try:
#                    string = [split[i- 4], split[i - 3],split[i-2], split[i-1], split[i], split[i+1], split[i + 2], split[i + 3], split[i + 4]]
#                    for strr in string:
#                        if strr.isdigit():
#                            print " ".join(string)
#                except:
#                    print d
                
    
print encuentros


4260


In [73]:
#Se recuperian bastante menos, es mas complicado recuperar los pisos

In [74]:
# Antes de agregar lo que vimos recien arreglamos un par de cosas que son que se tiene a barrios inexistentes en las distintas zonas como 
# barrio, ej: capital federal en CABA

In [75]:
barrios_caba = data[data["place_with_parent_names"].str.contains("Capital Federal")]["place_name"].value_counts().index

In [76]:
barrios_caba.delete(0) #Elimino capital federal

Index([u'Belgrano', u'Palermo', u'Caballito', u'Villa Urquiza',
       u'Villa Crespo', u'Flores', u'Barrio Norte', u'Almagro', u'Recoleta',
       u'Nuñez', u'Boedo', u'Balvanera', u'San Telmo', u'San Cristobal',
       u'Saavedra', u'Barracas', u'Colegiales', u'Villa del Parque',
       u'Floresta', u'Villa Devoto', u'Palermo Chico', u'Puerto Madero',
       u'Palermo Hollywood', u'Mataderos', u'Congreso', u'Monserrat',
       u'Villa Luro', u'Liniers', u'Villa Pueyrredón', u'Once',
       u'Parque Patricios', u'Palermo Soho', u'Coghlan', u'Parque Chacabuco',
       u'Centro / Microcentro', u'Chacarita', u'Las Cañitas', u'San Nicolás',
       u'Boca', u'Retiro', u'Villa Lugano', u'Villa Ortuzar', u'Constitución',
       u'Paternal', u'Parque Chas', u'Monte Castro', u'Parque Centenario',
       u'Abasto', u'Villa Santa Rita', u'Agronomía', u'Parque Avellaneda',
       u'Tribunales', u'Versalles', u'Palermo Viejo', u'Pompeya',
       u'Villa General Mitre', u'Villa Real', u'Velez Sarsf

In [77]:
barrios_zn = []
barrios_zo = []
barrios_zs = []
data_barrios = data["place_with_parent_names"]
for v in data_barrios:
   
    if v.split("|")[2] == "Bs.As. G.B.A. Zona Oeste":
        try:
            if v.split("|")[4] not in barrios_zo:
                barrios_zo.append(v.split("|")[4])
        except:
            if v.split("|")[3] not in barrios_zo:
                barrios_zo.append(v.split("|")[3])
    elif v.split("|")[2] == "Bs.As. G.B.A. Zona Sur":
        try:
            if v.split("|")[4] not in barrios_zs:
                barrios_zs.append(v.split("|")[4])
        except:
            if v.split("|")[3] not in barrios_zs:
                barrios_zs.append(v.split("|")[3])
    elif v.split("|")[2] == "Bs.As. G.B.A. Zona Norte":
        try:
            if v.split("|")[4] not in barrios_zn:
                barrios_zn.append(v.split("|")[4])
        except:
            if v.split("|")[3] not in barrios_zn:
                barrios_zn.append(v.split("|")[3])

In [78]:
barrios_zn.remove("")

In [79]:
barrios_zo.remove("")
barrios_zs.remove("")

In [80]:
#¿cuantos hay incorrectos?
bad_places = ["Capital Federal", "Bs.As. G.B.A. Zona Norte", "Bs.As. G.B.A. Zona Sur", "Bs.As. G.B.A. Zona Oeste"]
for place in bad_places:
    print len(data[data["place_name"] == place])

43712
12474
1034
1609


In [81]:
barrios = {"Capital Federal": barrios_caba, "Bs.As. G.B.A. Zona Oeste": barrios_zo, "Bs.As. G.B.A. Zona Sur":barrios_zs,
          "Bs.As. G.B.A. Zona Norte": barrios_zn}

In [82]:
def recuperar_barrio(row):
    place_actual = row["place_name"]
    #Si es nulo hacemos una busqueda mas exhaustiva 
    descripcion = row["description"]
    titulo = row["title"]
    extra = row["extra"]
    if (pd.isnull(place_actual)):
        zona = "Capital Federal"
        if row["place_with_parent_names"] == "Bs.As. G.B.A. Zona Norte":
            zona = "Bs.As. G.B.A. Zona Norte"
        
        elif row["place_with_parent_names"] == "Bs.As. G.B.A. Zona Sur":
            zona = "Bs.As. G.B.A. Zona Sur"
        elif row["place_with_parent_names"] == "Bs.As. G.B.A. Zona Oeste":
            zona = "Bs.As. G.B.A. Zona Oeste"
        barrios_posibles = barrios[zona]
        if not(pd.isnull(descripcion)):
            for b in barrios_posibles:
                if b in descripcion:
                    return b
       
    elif row["place_name"] in barrios:
        barrios_posibles = barrios[row["place_name"]]
        if not(pd.isnull(descripcion)):
            for b in barrios_posibles:
                if b in descripcion:
                    return b
       
    else:
        return row["place_name"]
        
            
        
            

In [83]:
data["place_name"] = data.apply(lambda row: recuperar_barrio(row), axis = 1)

/home/tomas/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [84]:
len(data[data["place_name"] == "Bs.As. G.B.A. Zona Norte"])

0

In [85]:
len(data[data["place_name"] == "Bs.As. G.B.A. Zona Sur"])

0

In [86]:
len(data[data["place_name"] == "Bs.As. G.B.A. Zona Oeste"])

0

In [87]:
len(data[data["place_name"] == "Capital Federal"])

0

In [88]:
# Recupero los pisos y cantidad de ambientes que podamos

In [89]:
def recuperar_atributo(row, atributo, info_atributo):
    atributo_act = row[atributo]
    if not(pd.isnull(atributo_act)):
        return atributo_act
    
    descripcion = row["description"]
    extra = row["extra"]
    title = row["title"]
    atributo_corregido = atributo_act
    cols = [descripcion, extra, title]
    for col in cols:
        if not(pd.isnull(col)):
            for k in info_atributo:
                if k in col:
                    atributo_corregido = info_atributo[k]
                    return atributo_corregido
    return atributo_corregido #Si no se logro encontro nunca, devuelvo el NaN
                    

In [90]:
data["floor"] = data.apply(lambda row: recuperar_atributo(row, "floor", posibles_descripciones_piso), axis = 1)

/home/tomas/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [91]:
data["rooms"] = data.apply(lambda row: recuperar_atributo(row, "rooms", posibles_descripciones_ambientes), axis = 1)

/home/tomas/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [92]:
data.isnull().sum()

                           803037
created_on                  42865
description                314330
expenses                   801025
extra                      568499
floor                      684553
geonames_id                164358
id                         591723
lat-lon                    247209
place_name                  58934
place_with_parent_names         0
price_aprox_usd                 0
price_usd_per_m2           171979
properati_url                   0
property_type                   0
rooms                      256032
state_name                 591723
surface_covered_in_m2      617936
surface_total_in_m2        171979
title                      314320
dtype: int64

In [95]:

data.to_csv("training_data.csv", compression = "gzip")